In [ ]:
from qibo.models import Circuit
from qibo import gates
import numpy as np
from tqdm.auto import tqdm
import lmfit
import matplotlib.pyplot as plt

import qililab as ql
from qililab.circuit_transpiler.native_gates import Wait
from qililab.calibration.calibration_node import export_nb_outputs


ql.logger.setLevel(40)

In [ ]:
qubit = 0
M_BUFFER_TIME = 0
HW_AVG = 3000
REPETITION_DURATION = 200_000
check = False
number_of_random_datapoints = 10
compare_fit = []
PLATFORM_PATH = "../../runcards/example_runcard.yml"
partition = "example_partition"

In [ ]:
# Define circuit
circuit = Circuit(qubit + 1)
circuit.add(ql.Drag(qubit, theta=np.pi, phase=0))
circuit.add(Wait(qubit, M_BUFFER_TIME))
circuit.add(gates.M(qubit))

In [ ]:
platform = ql.build_platform(runcard=PLATFORM_PATH)

IF = platform.get_parameter(alias=f"drive_q{qubit}_bus", parameter=ql.Parameter.IF)
sweep_interval = np.arange(IF - 5e6, IF + 5e6, step=0.2e6)
if len(compare_fit) > 0:
    compare_fit = [np.array(compare_fit[0]), np.array(compare_fit[1])]

if check and len(sweep_interval) != number_of_random_datapoints:
    sweep_interval = np.array(
        [sweep_interval[np.random.randint(0, len(sweep_interval))] for _ in range(number_of_random_datapoints)]
    )

old_amplitude = platform.get_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.AMPLITUDE)
old_duration = platform.get_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.DURATION)

platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.AMPLITUDE, value=0.005)
platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.DURATION, value=1000)

In [ ]:
job_name = f"AC-2T-q{qubit}"

In [ ]:
%%submit_job -o results -p $partition -n $job_name

platform.connect()
platform.initial_setup()
platform.turn_on_instruments()

# Run experiment
results = []
for freq in tqdm(iterable=sweep_interval, total=len(sweep_interval), desc="IF frequency", colour="green"):
    platform.set_parameter(alias=f"drive_q{qubit}_bus", parameter=ql.Parameter.IF, value=float(freq))
    result = platform.execute(program=circuit, num_avg=HW_AVG, repetition_duration=REPETITION_DURATION)
    results.append(result.array)

platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.AMPLITUDE, value=old_amplitude)
platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.DURATION, value=old_duration)

results = np.hstack(results)

platform.disconnect()

In [ ]:
results = results.result()
loops = {"drive_q{qubit}_bus_if": sweep_interval}
ql.save_results(results=results, loops=loops, data_path="../data/", name=job_name)

In [ ]:
def plot_iq(xdata, results: np.ndarray, title_label: str, xlabel: str, check=False):
    fig, axes = plt.subplots(1, 2, figsize=(13, 7))
    if check:
        axes[0].scatter(xdata, results[0])
        axes[1].scatter(xdata, results[1])
    else:
        axes[0].plot(xdata, results[0], "--o", color="blue")
        axes[1].plot(xdata, results[1], "--o", color="blue")
    axy = axes[1].twiny()
    # other_xlim = axes[1].get_xlim()
    axy.set_xlim(-10, 10)
    axes[0].set_title("I")
    axes[1].set_title("Q")
    axes[0].set_xlabel(xlabel)
    axes[1].set_xlabel(xlabel)
    axes[0].set_ylabel("Voltage [a.u.]")
    axes[1].set_ylabel("Voltage [a.u.]")
    fig.suptitle(title_label)
    return fig, axes

In [ ]:
# Initialize dictionaries
r_squared = {"i": 0, "q": 0}
fitted_ifs = {"i": None, "q": None}
fitted_eval_funcs = []

# Loop over quadratures
for fit_quadrature in ["i", "q"]:
    # Fitting function
    def lorentzian(x, amplitude, center, width, offset):
        return amplitude / (1 + ((x - center) / (0.5 * width)) ** 2) + offset

    # Fit signal
    fit_signal = results[0] if fit_quadrature == "i" else results[1]

    # Lorentzian fit
    mod = lmfit.Model(lorentzian)

    # Set initial parameter values
    initial_amp = max(fit_signal) - min(fit_signal)
    if (np.mean(fit_signal) - min(fit_signal)) ** 2 > (np.mean(fit_signal) - max(fit_signal)) ** 2:
        # if the mean of the values is closer to the maximum, it means we have negative amplitude!
        initial_amp = -initial_amp

    mod.set_param_hint(
        "amplitude",
        value=initial_amp,
        min=-abs(max(fit_signal) - min(fit_signal)),
        max=abs(max(fit_signal) - min(fit_signal)),
    )
    mod.set_param_hint("center", value=np.mean(sweep_interval), min=np.min(sweep_interval), max=np.max(sweep_interval))
    mod.set_param_hint(
        "width",
        value=(max(sweep_interval) - min(sweep_interval)) / 10,
        min=(max(sweep_interval) - min(sweep_interval)) / 100,
        max=(max(sweep_interval) - min(sweep_interval)),
    )
    mod.set_param_hint("offset", value=np.mean(fit_signal), min=np.min(fit_signal), max=np.max(fit_signal))

    params = mod.make_params()
    fit = mod.fit(data=fit_signal, params=params, x=sweep_interval, method="differential_evolution")

    # Update r_squared value
    r_squared[fit_quadrature] = fit.rsquared

    fitted_amplitude = fit.params["amplitude"].value
    fitted_center = fit.params["center"].value
    fitted_width = fit.params["width"].value
    fitted_offset = fit.params["offset"].value

    popt = [fitted_amplitude, fitted_center, fitted_width, fitted_offset]

    xdata = np.linspace(min(sweep_interval), max(sweep_interval), num=1000)

    if fitted_amplitude < 0:
        fitted_if = sweep_interval[
            np.argmin(lorentzian(sweep_interval, fitted_amplitude, fitted_center, fitted_width, fitted_offset))
        ]
    else:
        fitted_if = sweep_interval[
            np.argmax(lorentzian(sweep_interval, fitted_amplitude, fitted_center, fitted_width, fitted_offset))
        ]

    fitted_ifs[fit_quadrature] = fitted_if
    fitted_func = lorentzian(sweep_interval, *popt)
    fitted_eval_funcs.append(fitted_func)

    # Plot
    fig, axes = plot_iq(
        xdata=sweep_interval, results=results, title_label=f"2 Tone q{qubit}", xlabel="Amplitude", check=check
    )
    ax = axes[0 if fit_quadrature == "i" else 1]
    if check and len(compare_fit) > 0:
        fitted_func = compare_fit[1][0] if fit_quadrature == "i" else compare_fit[1][1]
        ax.plot(compare_fit[0], fitted_func, "--", label="previously calibrated fit", color="red")
        ax.legend()
    else:
        label_fit = f"FITTED IF = {fitted_if}"
        ax.plot(sweep_interval, fitted_func, "--", label=label_fit, color="red")
    ax.legend()

# Display results or further analysis as needed

In [ ]:
best_quadrature = "i" if r_squared["i"] > r_squared["q"] else "q"
best_fit = fitted_ifs[best_quadrature]

print(f"Fitted IF: {best_fit}")

In [ ]:
key = "IF"
print(f"OUTPUTS{key}:{best_fit}")

In [ ]:
platform_parameters = [(ql.Parameter.IF, float(best_fit), f"drive_q{qubit}_bus", None)]

In [ ]:
export_nb_outputs(
    {
        "check_parameters": {
            "sweep_interval": sweep_interval,
            "results": results,
            "fit": fitted_eval_funcs,
            "optimal_parameter": best_fit,
        },
        "platform_parameters": platform_parameters,
    }
)